In [1]:
import geopandas as gpd
def code_vege(shapefile_path_vege,shapefile_path_emprise,output_path):
   
    # Charger le fichier shapefile
    gdf_vegetation = gpd.read_file(shapefile_path_vege)
    gdf_emprise = gpd.read_file(shapefile_path_emprise)
    
    # Ajouter les champs "nom" et "code" avec des valeurs par défaut
    gdf_vegetation['nom'] = None
    gdf_vegetation['code'] = None
    
   
    # Création d'un dictionnaire pour mapper les valeurs de 'TFV' aux colonnes 'nom' et 'code'
    mapping = {
        "FF1-49-49": ("Autres feuillus", "11"),
        "FF1-10-10": ("Autres feuillus", "11"),
        "FF1-09-09": ("Autres feuillus", "11"),
        "FF1G01-01": ("chêne", "12"),
        "FF1-14-14": ("Robinier", "13"),
        "FP": ("Peupleraie", "14"),
        "FF1-00-00": ("Mélange de feuillus", "15"), 
        "FF1-00": ("Feuillus en îlots", "16"),
        "FF2-91-91": ("Autres conifères autre que pin", "21"),
        "FF2-63-63": ("Autres conifères autre que pin", "21"),
        "FF2G61-61": ("Autres conifères autre que pin", "21"),
        "FF2-90-90": ("Autres conifères autre que pin", "21"),
        "FF2-81-81": ("Autres Pin", "22"),
        "FF2-52-52": ("Autres Pin", "22"),
        "FF2-80-80": ("Autres Pin", "22"),
        "FF2-64-64": ("douglas", "23"),
        "FF2G53-53": ("Pin laricio ou pin noir", "24"),
        "FF2-51-51": ("Pin maritime", "25"),
        "FF2-00-00": ("Mélange conifères", "26"),
        "FF2-00": ("Conifères en îlots", "27"),
        "FF32": ("Mélange de conifères prépondérants et feuillus", "28"),
        "FF31": ("Mélange de feuillus prépondérants et conifères", "29"),
    }
    
    # Application des valeurs du mapping
    gdf_vegetation['nom'] = gdf_vegetation['CODE_TFV'].map(lambda x: mapping[x][0] if x in mapping else None)
    gdf_vegetation['code'] = gdf_vegetation['CODE_TFV'].map(lambda x: mapping[x][1] if x in mapping else None)
    
    gdf_vegetation = gdf_vegetation.dropna(subset=['code'])
    
    
    vege_emprise = gpd.clip(gdf_vegetation, gdf_emprise)
    # Sauvegarder les modifications dans un nouveau fichier (ou remplacer l'original)
    
    vege_emprise.to_file(output_path)

In [3]:
shapefile_path_vege = '../data/project/FORMATION_VEGETALE.shp'
shapefile_path_emprise = '../data/project/emprise_etude.shp'
output_path = 'Sample_BD_foret_T31TCJ.shp'
code_vege(shapefile_path_vege,shapefile_path_emprise,output_path)

In [5]:
import sys
sys.path.append('../libsigma/')
import os
import read_and_write as rw
def rasterize(in_vector,out_image,field_name,sptial_resolution):

  # define command pattern to fill with paremeters
  cmd_pattern = ("gdal_rasterize -a {field_name} "
                 "-tr {sptial_resolution} {sptial_resolution} "
                 "-ot Byte -of GTiff "
                 "-a_nodata 0 "
                 "{in_vector} {out_image}")
  
  # fill the string with the parameter thanks to format function
  cmd = cmd_pattern.format(in_vector=in_vector, out_image=out_image, field_name=field_name,
                           sptial_resolution=sptial_resolution)
  
  # execute the command in the terminal
  print(cmd)
  os.system(cmd)

In [7]:
in_vector = 'Sample_BD_foret_T31TCJ.shp'
out_image = 'raster.tif'
field_name = 'code'
sptial_resolution = 10.0
rasterize(in_vector,out_image,field_name,sptial_resolution)

gdal_rasterize -a code -tr 10.0 10.0 -ot Byte -of GTiff -a_nodata 0 Sample_BD_foret_T31TCJ.shp raster.tif
0...10...20...30...40...50...60...70...80...90...100 - done.


In [8]:
def masque(input_image,output_masque):
    data_set = rw.open_image(input_image)
    img = rw.load_img_as_array(input_image)
    
    
    # do some processingwith numpy: build a mask
    nb_ligne, nb_col, nb_band = rw.get_image_dimension(data_set)
    masque_foret = img != 0
    
    # write it
    rw.write_image(output_masque, masque_foret, data_set=data_set,
                gdal_dtype=data_type_match['uint8'])

In [9]:
from osgeo import gdal
data_type_match = {'uint8': gdal.GDT_Byte,
                   'uint16': gdal.GDT_UInt16,
                   'uint32': gdal.GDT_UInt32,
                   'int16': gdal.GDT_Int16,
                   'int32': gdal.GDT_Int32,
                   'float32': gdal.GDT_Float32,
                   'float64': gdal.GDT_Float64}
input_image = 'raster.tif'
output_masque = 'masque_foret.tif'
masque(input_image,output_masque)

/opt/conda/lib/python3.12/site-packages/osgeo/gdal.py:312: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(
